#  Data Understanding & Exploration

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv('/kaggle/input/top-100-cryptocurrency-2020-2025/top_100_cryptos_with_correct_network.csv')

In [3]:
print("Dataset shape:", df.shape)
print("Column names:", df.columns.tolist())

Dataset shape: (211679, 7)
Column names: ['symbol', 'date', 'open', 'high', 'low', 'close', 'network']


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211679 entries, 0 to 211678
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   symbol   211679 non-null  object 
 1   date     211679 non-null  object 
 2   open     211679 non-null  float64
 3   high     211679 non-null  float64
 4   low      211679 non-null  float64
 5   close    211679 non-null  float64
 6   network  211679 non-null  object 
dtypes: float64(4), object(3)
memory usage: 11.3+ MB


In [5]:
df.head()

,symbol,date,open,high,low,close,network
0,BTCUSDT,2018-08-01,7735.67,7750.00,7430.00,7604.58,Bitcoin
1,BTCUSDT,2018-08-02,7600.08,7709.46,7455.72,7525.71,Bitcoin
2,BTCUSDT,2018-08-03,7525.71,7540.00,7282.44,7418.78,Bitcoin
3,BTCUSDT,2018-08-04,7412.27,7494.81,6926.00,7009.84,Bitcoin
4,BTCUSDT,2018-08-05,7009.84,7089.87,6882.29,7024.19,Bitcoin


In [7]:
df.tail()

,symbol,date,open,high,low,close,network
211674,UMAUSDT,2025-08-01,1.242,1.251,1.162,1.202,Ethereum
211675,UMAUSDT,2025-08-02,1.202,1.216,1.147,1.172,Ethereum
211676,UMAUSDT,2025-08-03,1.175,1.210,1.158,1.207,Ethereum
211677,UMAUSDT,2025-08-04,1.206,1.247,1.201,1.242,Ethereum
211678,UMAUSDT,2025-08-05,1.243,1.245,1.204,1.234,Ethereum


In [8]:
print(df.isnull().sum())


symbol     0
date       0
open       0
high       0
low        0
close      0
network    0
dtype: int64


In [11]:
display(df.describe(include='all'))


,symbol,date,open,high,low,close,network
count,211679,211679,211679.000000,211679.000000,211679.000000,211679.000000,211679
unique,100,2562,NaN,NaN,NaN,NaN,50
top,BTCUSDT,2022-02-02,NaN,NaN,NaN,NaN,Ethereum
freq,2562,100,NaN,NaN,NaN,NaN,94654
mean,NaN,NaN,628.566155,646.550242,609.654246,629.092910,NaN
std,NaN,NaN,5345.009524,5478.052232,5207.376818,5350.691462,NaN
min,NaN,NaN,0.000043,0.000044,0.000037,0.000043,NaN
25%,NaN,NaN,0.061740,0.064500,0.059095,0.061750,NaN
50%,NaN,NaN,0.418100,0.438400,0.398000,0.418000,NaN
75%,NaN,NaN,4.383000,4.595200,4.165000,4.383000,NaN
